# 波动率计算与应用

## 本段代码利用quantOS系统分析了三个期货产品日收益率的波动率，及其能否随时间变化scale

In [ ]:
# import sys
# sys.path.append('/home/bliu/pytrade_dir/jaqs')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from jaqs.data import RemoteDataService
sns.set_style('darkgrid')
%matplotlib inline

### 系统设置

In [ ]:
# 设置服务器地址、用户名密码
# 例如：
# data_config = {
#   "remote.data.address": "tcp://data.tushare.org:8910",
#   "remote.data.username":  '18688888888',
#   "remote.data.password":  '23sdjfk209d0fs9dejkl2j3k4j9d0fsdf'}

# 如果没有使用quantos金融终端，请自行替换phone,token内容
import os
phone = os.environ.get("QUANTOS_USER")
token = os.environ.get("QUANTOS_TOKEN")

data_config = {
  "timeout": 1800,
  "remote.data.address": "tcp://data.tushare.org:8910",
  "remote.data.username":  phone,
  "remote.data.password":  token}

# 启动DataView，并设置用户名密码
ds = RemoteDataService()
ds.init_from_config(data_config)

### 整合连续合约（可忽略这部分的实现细节）

In [ ]:
def query_daily_data(data_service, df_inst_map_, fields_):
    future_list = []
    for _, row in df_inst_map_.iterrows():
        startdate, enddate, symbol = int(row['pre_trade_date']), row['trade_date'], row['symbol']
        df_future, msg1 = data_service.daily(symbol, start_date = startdate, end_date = enddate, fields=fields_)
        df_future['pre_close'] = df_future['close'].shift(1)
        df_future['log_ret'] = np.log(df_future['close']).diff()
        df_future = df_future.dropna()
        future_list.append(df_future)
    return future_list

In [ ]:
def get_symbol_date_map(data_service, df_inst_, start_date, end_date, days_to_delist):
    """
    Get a map {trade_date -> front month contract symbol}

    """
    # get the trade date list between start_date and end_date
    dates = data_service.get_trade_date_range(start_date, end_date)
    symbols_list = []
    move_list = []

    mask_within_range = (df_inst_['delist_date'] > dates[0]) & (df_inst_['list_date'] < dates[-1])
    df_inst_ = df_inst_.loc[mask_within_range]

    j = 0
    for i, td in enumerate(dates):
        delist_date = df_inst_['delist_date'].iat[j]
        idx = np.nonzero(dates == delist_date)

        if (delist_date <= dates[-1]) and (idx[0][0] - i <= days_to_delist):
            j += 1
            delist_date = df_inst_['delist_date'].iat[j]
            move_list.append(1)
        else:
            move_list.append(0)
        symbol = df_inst_['symbol'].iat[j]
        symbols_list.append(symbol)

    res = pd.DataFrame(data={'trade_date': dates, 'symbol': symbols_list, 'move': move_list})
    res.loc[:, 'move'] = res['move'].shift(-1)
    return res

In [ ]:
def get_continueContract(FUTURE_SYMBOL, MARKET, START_DATE, END_DATE, CHANGE_DATE, FIELDS):

    # get the information of front month contract
    df_inst = ds.query_inst_info(symbol="",
                                 fields=','.join(['symbol', 'inst_type', 'market', 'status', 'multiplier', 'list_date',
                                                  'delist_date']))
    df_inst = df_inst.reset_index()
    mask_stock_index_future = df_inst['symbol'].apply(lambda s:
                                                      s.startswith(FUTURE_SYMBOL) and s.endswith(MARKET) and s[len(FUTURE_SYMBOL)] != 'F'
                                                      )
    df_inst = df_inst.loc[mask_stock_index_future]
    df_inst.index = range(len(df_inst))
    df_inst = df_inst.sort_values('delist_date')

    first_list_date = df_inst['list_date'].min()
    if START_DATE < first_list_date:
        START_DATE = first_list_date
    else:
        pass

    # get the front month contract for each trading date
    df_inst_map = get_symbol_date_map(ds, df_inst, START_DATE, END_DATE, CHANGE_DATE)
    df_inst_map['pre_trade_date'] = df_inst_map['trade_date'].shift(1)
    df_inst_map = df_inst_map.dropna()
    df_inst_map['pre_trade_date'] = df_inst_map['pre_trade_date'].apply(lambda x: int(x))

    # get the daily info
    symbols = np.unique(df_inst_map['symbol'])
    symbols_str = ','.join(symbols)

    future_list = query_daily_data(ds, df_inst_map, FIELDS)

    df_future_daily = pd.concat(future_list).astype({'trade_date': np.integer}).set_index(
        ['trade_date'])
    del df_future_daily['freq']
    df_inst_map.index = df_inst_map['trade_date']
    df_future_daily = pd.merge(left = df_future_daily, right = df_inst_map[['move']],
                               left_index=True, right_index=True, how = 'left')

    df_future_daily['ret_shift'] = 0.0

    df_future_daily['trade_date'] = df_future_daily.index
    df_future_daily.index = range(len(df_future_daily))

    for i in range(len(df_future_daily) - 1):
        if df_future_daily.ix[i, 'move'] == 1.0:
            df_future_daily.ix[i, 'ret_shift'] = np.log(df_future_daily.ix[i+1, 'close']) - np.log(df_future_daily.ix[i, 'close'])

    df_future_daily.index = df_future_daily['trade_date']
    df_future_daily['trade_date'] = df_future_daily['trade_date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d').date())
    return df_future_daily

### 设置参数

In [ ]:
# 开始日期
STARTDATE = 20130101
# 结束日期
ENDDATE   = 20180108
# 到期日前n天进行合约切换
DAYS_BEFORE_SHIFT = 3
RollingWindow = 30
FIELDS = 'close'

### 取期货连续合约日收益率

In [ ]:
IF_df = get_continueContract('IF', 'CFE', STARTDATE, ENDDATE, DAYS_BEFORE_SHIFT, FIELDS)
T_df = get_continueContract('T', 'CFE', STARTDATE, ENDDATE, DAYS_BEFORE_SHIFT, FIELDS)
rb_df = get_continueContract('rb', 'SHF', STARTDATE, ENDDATE, DAYS_BEFORE_SHIFT, FIELDS)

### 做图展示三个品种的日对数收益率 

In [ ]:
# IF
fig, ax1 = plt.subplots(figsize=(18, 9))
ax1.plot(IF_df['trade_date'], IF_df['log_ret'], label = 'log return', color = 'blue', linewidth = 0.8, alpha = 0.8)
ax1.axhline(y = 0, color = 'red', linestyle = '--', linewidth = 0.8, alpha = 0.8)
plt.legend(fontsize = 12)
plt.title('Daily log return of IF contract', fontsize=15)
plt.show()

In [ ]:
# T
fig, ax1 = plt.subplots(figsize=(18, 9))
ax1.plot(T_df['trade_date'], T_df['log_ret'], label = 'log return', color = 'blue', linewidth = 0.8, alpha = 0.8)
ax1.axhline(y = 0, color = 'red', linestyle = '--', linewidth = 0.8, alpha = 0.8)
plt.legend(fontsize = 12)
plt.title('Daily log return of T contract', fontsize=15)
plt.show()

In [ ]:
# rb
fig, ax1 = plt.subplots(figsize=(18, 9))
ax1.plot(rb_df['trade_date'], rb_df['log_ret'], label = 'log return', color = 'blue', linewidth = 0.8, alpha = 0.8)
ax1.axhline(y = 0, color = 'red', linestyle = '--', linewidth = 0.8, alpha = 0.8)
plt.legend(fontsize = 12)
plt.title('Daily log return of rb contract', fontsize=15)
plt.show()

### 计算移动窗口波动率

In [ ]:
# 计算波动率
def calc_vol(arr):
    return np.sqrt(np.sum((arr - 0.0)**2) / len(arr))

In [ ]:
# 计算移动窗口波动率
def process_roll_vol(df):
    df['rolling_vol'] = df['log_ret'].rolling(window=RollingWindow).apply(calc_vol)
    return df
def plot_roll_vol(df, name):
    fig, ax1 = plt.subplots(figsize=(18, 9))
    ax1.plot(df['trade_date'], df['rolling_vol'], label = 'Normal Std', color = 'blue', linewidth = 0.8, alpha = 0.8)
    #ax1.plot(df['trade_date'], df['ewma_vol'], label = 'EWMA Std', color = 'red', linewidth = 0.8, alpha = 0.8)
    # ax1.axhline(y = 0, color = 'red', linestyle = '--', linewidth = 0.8, alpha = 0.8)
    plt.legend(fontsize = 12)
    plt.title('Rolling window volatility of {} contract'.format(name), fontsize=15)
    plt.show()

In [ ]:
IF_df = process_roll_vol(IF_df)
plot_roll_vol(IF_df, 'IF')

In [ ]:
T_df = process_roll_vol(T_df)
plot_roll_vol(T_df, 'T')

In [ ]:
rb_df = process_roll_vol(rb_df)
plot_roll_vol(rb_df, 'RB')

### 检验波动率可否scale

In [ ]:
# periodList = [1,2,3,4,5,10,20,40,50,60,70,80,90,100]
periodList = np.arange(1,40)
TO_BPS = 1e4

In [ ]:
def scale_plot(dframe, symbol):
    volList = []
    for Nperiod in periodList:
        dframe['group'] = np.arange(len(dframe))/Nperiod
        dframe_cum_ret = dframe.groupby('group')['log_ret'].sum()
        vol = calc_vol(dframe_cum_ret)
        volList.append(vol)
    scalevolList = [np.sqrt(x) * volList[0] for x in periodList]
    
    table = pd.DataFrame(data={'n Days': periodList, 'Volatility (bps)': volList}).set_index('n Days')
    table = table.T
    table = (table * TO_BPS).astype(int)
    
    # 画图
    fig, ax1 = plt.subplots(figsize=(18, 9))
    ax1.scatter(periodList, volList)
    ax1.plot(periodList, scalevolList, color = 'red', linestyle = '-', linewidth = 0.8, alpha = 0.8,
            label = r'$\sqrt{x}$')
    ax1.set_ylabel('volatility', fontsize = 15)
    ax1.set_xlabel('days', fontsize = 15)
    plt.legend(fontsize = 15)
    plt.title('Scalability of volatlity of %s contract' % symbol, fontsize=15)
    plt.show()
    
    return table.iloc[:, [0, 4, 9, 19, 29]]

In [ ]:
scale_plot(T_df, 'T')

In [ ]:
scale_plot(IF_df, 'IF')

In [ ]:
scale_plot(rb_df, 'RB')